# TO DO
# ----------------
  - Options for p-type, ambipolar, and n-type
  - Display filename that is open *NOTE: cannot be done in Dash*
  - Make filename entry box longer
  - FIGURE OUT ambipolar calculations??

# NOTES
# ------------------------------------------------
- Ideal curve fitting isn't working, seems like selectedData isn't getting passed into selected_Vg or selected_Id
- r values are being calculated, and they're different each time, so maybe it's a graphing issue, not a calculation
- Adding in the p, n, ambi dropdown and input messed it all up. Amy mentioned having to switch to State variables to
  avoid a high number of Inputs
- Need to delve into State variables a bit more

In [1]:
from textwrap import dedent as d
import numpy as np
from scipy import stats
import pandas as pd

import base64
import io
import os
from decimal import Decimal

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State

In [2]:
app = dash.Dash()

In [3]:
# Device specific characteristics. Corresponds to Shadowmasks purchased by WKT in May, 2018 on substrate of heavily
# Boron-doped Si with 300 nm of thermally grown SiO2

# All equations and figures of merit are based on the following papers:

# Chem. Mater., 2015, 27 (12), pp 4167–4168
# Nature Materials volume 17, (2018) pages 2–7

L = 50        #50 micron channel length
W = 1000      #1000 micron channel width
Ci = 11.5e-9  #11.5 nF/cm^2

In [4]:
# Define graph layout for linear regime calculations

ptype_layout = go.Layout(
    xaxis={'title': 'Vg (V)'},
    yaxis={'title': 'Id (A)'},
    height=350,
    margin=go.Margin(
        l=35,
        r=5,
        b=40,
        t=5),
    dragmode='select',
    legend=dict(
        borderwidth=2,
        xanchor='right')
)

ntype_layout = go.Layout(
    xaxis={'title': 'Vg (V)'},
    yaxis={'title': 'Id (A)'},
    height=350,
    margin=go.Margin(
        l=35,
        r=5,
        b=40,
        t=5),
    dragmode='select',
    legend=dict(
        borderwidth=2,
        xanchor='right')
)

ambipolar_layout = go.Layout(
    xaxis={'title': 'Vg (V)'},
    yaxis={'title': 'Id (A)'},
    height=350,
    margin=go.Margin(
        l=35,
        r=5,
        b=40,
        t=5),
    dragmode='select',
    legend=dict(
        borderwidth=2,
        xanchor='right')
)

/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [5]:
# Format page, reference utilities, markdown explanations

app.layout = html.Div([
    html.H1(html.B('Transfer Curve Analysis')),
    
    html.Div([
        html.Div([dcc.Markdown(d('''
            - Ensure that the file is a .txt format file that only has one table within it
            - Gate Voltage should be in a column titled 'Vg(V)', Absolute Drain Current titled '-Id(A)', and Sqrt Drain Current titled 'Sqrt(Id)'
            - If Vd > Vg-Vt, (e.g. Vd = -100 V), then you are calculating saturated mobility
            - If Vd << Vg, (e.g. Vd = -10 V), then you are calculating linear mobility
            - Threshold voltage (Vt) is calculated from transfer curve in the Saturated Regime''')),
            html.Div([html.B(html.I(
                'Linear mobility is the preferred value for publications'
                ))], 
                style = {
                    'fontSize': 20,
                    }),
            ],
        style = {
            'maxWidth': 900,
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '20px',
            'borderColor': 'rgb(160, 160, 160)',
            'padding': 20,
            'margin': '0px auto'
        })
    ]),
    
    html.H3('Once all transfer curves for a device have been analyze and the values are appended to the above file, append the averages to the end of the SAME file'),
    
    html.Hr(),
    
    html.Div([
        html.Div([
            html.H3('Vg Selection'),    
            html.Div('Select the range of Vg, where the curve is linear, using the Box Select Tool below:'),
            dcc.Graph(
                id='transfer-curve',
                figure=go.Figure(),
                selectedData={'points': [], 'range': None}
            )
        ], style={'width': '50%', 'display': 'inline-block'}),
        
        html.Div([
            html.H3('Linear Regression'),
            html.Div('After selection, linear fit of the specified region and of the electronically equivalent ideal FET will be generated below:'),
            dcc.Graph(id='linear-fit',
                      figure=go.Figure())
            
        ],
            style={'width': '50%', 'display': 'inline-block'}
        ),
    ],
        style = {'width': '100%',
                'display': 'inline-block'}
    ),
    

    html.Div(id='selected-data', style={'fontSize': 20}),
    
    html.Hr(),
    
    html.Div([
        html.Div([
            dcc.Upload(id = 'upload-data',
                       children = html.Button('Upload File')
                      ),
            
            html.Hr(),
            
            html.Div([
                html.Label('Enter the voltage, Vd, that was used to generate the curve:'),
                dcc.Input(
                    id = 'Vd',
                    placeholder = 'Enter Vd...',
                    type = 'number',
                    value = -20
                    ),
            ]),
            
            html.Hr(),
            
            html.Div([
                html.Label('Semiconductor type:'),
                dcc.Dropdown(
                    id = 'conduction',
                    options = [
                        {'label': 'p-type', 'value':'p'},
                        {'label': 'ambipolar', 'value':'ambi'},
                        {'label': 'n-type', 'value':'n'}
                    ],
                    value = 'p'
                ),
            ],
            style = {'width': '30%',
                    'display': 'inline-block'
                    }
            ),
        ],
        style = {'width': '30%',
                'display': 'inline-block'
                }
        ),
        
        html.Div([
            html.Label('Enter the desired name of your output file, with .txt at the end:'),
            dcc.Input(
                id='output-filename',
                type='text',
                value='filename.txt'
            ),
            
            html.Hr(),

            html.Div(id='filename'),

            html.Button(id='submit-button', n_clicks=0, children='Calculate'),

            html.Div(id='output1'),
            
            html.Hr(),
            
            dcc.Upload(id='average-data', children=html.Button('Average Me!')),
            html.Div(id='average'),
        ],
            style = {'width': '30%',
                    'display': 'inline-block'}
        ),
        ],
            style = {
                'width': '100%',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '20px',
                'borderColor': 'rgb(160, 160, 160)',
                'padding': 10,
                'margin': '0px auto',
                'display': 'inline-block'
                }
        ),

        html.Hr(),
    ], 
        style={
            'textAlign': 'center',
            'margin': '10px 30px'
        }
    )

In [6]:
# Template for compiling and designing webpage

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

# -------------------------------------------------------------------------------------------

In [7]:
# Accesses uploaded data and defines 3 dataframes, corresponding to Vg, |Id|, and sqrt(Id)

def dataframe(contents):
    
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df = pd.read_table(io.StringIO(decoded.decode('utf-8')))

    df_Vg = df['Vg(V)'] 
    df_Id = df['Id(A)']
    df_Id1 = df['-Id(A)']
    df_sqrtId = df['Sqrt(Id)']
    
    Vg = df_Vg.values
    Id = df_Id.values
    sqrtId = df_sqrtId.values
    
    negId = np.zeros(len(Id))
    
    negId = Id*(-1)
    
    return Vg, Id, negId, sqrtId

In [8]:
# Determines the ideal, Shockley curve

def idealreg_linear(contents, conduction = 'p'):
    Vg, Id, negId, sqrtId = dataframe(contents)
    ideal_Id = [_]
    ends_Vg = [_,_]
    ends_Id = [_,_]
    
    ends_Vg[0] = min(Vg)
    ends_Vg[1] = max(Vg)
    
    if conduction == 'n':
        ends_Id[0] = min(Id)
        ends_Id[1] = max(Id)
            
    elif conduction == 'p':
        ends_Id[0] = min(negId)
        ends_Id[1] = max(negId)
            
    else:
        ends_Id[0] = min(Id)
        ends_Id[1] = max(Id)
    
    ideal_slope, ideal_intercept, r_value, p_value, std_err = stats.linregress(ends_Vg, ends_Id) ## ideal fit for Id
    
    ideal_Id.append((ideal_slope * Vg) + ideal_intercept)
    
    return ideal_slope, ideal_intercept, ideal_Id

In [9]:
# Given the selected data points in the linear regime, perform linear regression and determine
# properties from regression results

def calculate_linear_output(contents, selectedData, Vd, conduction_type = 'p'):
    Vg, Id, negId, sqrtId = dataframe(contents)
    conduction = conduction_type
    
    selected_Vg = []
    selected_Id = []
    
    for i in range(len(selectedData['points'])):  
        selected_Vg.append(selectedData['points'][i]['x'])
        selected_Id.append(selectedData['points'][i]['y'])
    
    Id_slope, Id_intercept, r_value, p_value, std_err = stats.linregress(selected_Vg, selected_Id)
    
    ideal_slope, ideal_intercept, ideal_Id = idealreg_linear(contents, conduction)
    
    mu_lin = (Id_slope* 1 * L) / (Vd * W * Ci)
    r_lin = ideal_slope / Id_slope
    
    if conduction == 'n':
        Id_max = max(Id)
        Id_min = min(Id)
    
    elif conduction == 'p':
        Id_max = max(negId)
        Id_min = min(negId)
        
    else:
        Id_max = max(negId)
        Id_min = min(negId)

    on_off = Id_max/Id_min
    Vt = -Id_intercept/Id_slope
    
    values = np.array([mu_lin, r_lin, on_off, Vt])
    
    return values

In [10]:
# Displays uploaded data

@app.callback(Output('transfer-curve', 'figure'),
             [Input('upload-data', 'contents')])
def display_uploaded_data(contents):
    Vg, Id, negId, sqrtId = dataframe(contents)
    
    yy = Id
    a_layout = ptype_layout
    
    return go.Figure(
        data=[
            go.Scatter(
                x=Vg,
                y=yy,
                mode='lines+markers'
            )
        ],
        layout=a_layout)

In [11]:
# Prints selected range from the graph object below graphs

@app.callback(
    Output('selected-data', 'children'),
    [Input('transfer-curve', 'selectedData')])
def display_selected_data(selectedData):
    return 'Your selected range is Vg: (', int(selectedData['range']['x'][0]), ', ', int(selectedData['range']['x'][1]), ')'

In [12]:
# Creates linear regression of selected region

@app.callback(
    Output('linear-fit', 'figure'),
     [Input('upload-data', 'contents'),
      Input('transfer-curve', 'selectedData')])
#      [State('conduction', 'value')])
def create_linreg(contents, selectedData):
    Vg, Id, negId, sqrtId = dataframe(contents)
#     conduction = conduction
    
    selected_Vg = []
    selected_Id = []
    fit_Id = []
    
    for i in range(len(selectedData['points'])):
        selected_Vg.append(selectedData['points'][i]['x'])
        selected_Id.append(selectedData['points'][i]['y'])
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(selected_Vg,selected_Id)
    
    fit_Id.append(slope * Vg + intercept)
        
    _, _, ideal_Id = idealreg_linear(contents)
    
#     if conduction == 'n':
    Id_data = Id
#     elif conduction == 'p':
#         Id_data = negId
#     else:
#         Id_data = negId
        
    IdLabel = 'Id'
    ideal_Id_data = ideal_Id
    a_layout = ptype_layout
    
    return {
        'data': [
            go.Scatter(
                x=Vg,
                y=Id_data,
                name='{} vs Vg'.format(IdLabel),
                line=dict(
                    color=('rgb(0, 0, 0)'),
                    width=3)),
            go.Scatter(
                x=Vg,
                y=fit_Id,
                name='Fit',
                line=dict(
                    color=('rgb(255, 8, 0)'),
                    dash='dash')),
            go.Scatter(
                x=Vg,
                y=ideal_Id_data,
                name='Ideal',
                line=dict(
                    color=('rgb(0, 0, 255)'),
                    dash='dash'))],
        'layout': a_layout
    }

In [13]:
# Take in and display filename information for output file

@app.callback(
    Output('filename', 'children'),
    [Input('output-filename', 'value')]
)
def update_output_div(input_value):
    return 'Calculated values will be saved to the file "{}"'.format(input_value)

In [14]:
# Retrieves outputs for linear mobility

@app.callback(
    Output('output1', 'children'),
    [Input('submit-button', 'n_clicks')],
#     [State('conduction', 'value'),
    [State('upload-data', 'contents'),
     State('transfer-curve', 'selectedData'),
     State('Vd', 'value'),
     State('output-filename', 'value')]
)
def calculate_output(n_clicks, contents, selectedData, Vd, filename):
    
#     conduct = conduction
    
    return_text = ['mu_lin ', ' r_lin ', ' On-Off Ratio ', ' Vt ']
    total_return = []
    mu_lin = 0
    r_lin = 0
    on_off = 0
    mu_sat = 0
    r_sat = 0
    Vt = 0
    
    values = calculate_linear_output(contents, selectedData, Vd)
    
    for i in range(len(return_text)):
        output = "{:.2E}".format(Decimal(values[i]))
        total_return.append(f"{return_text[i]} = {output}")
    
    if os.path.exists(filename): 
        output_file = np.genfromtxt(filename)
        final_file = np.vstack((output_file, values))
        np.savetxt(filename, final_file, delimiter=" ", fmt="%s", header='mu_lin, r_lin, on/off, Vt')
        
    else:
        np.savetxt(filename, values, delimiter=" ", fmt="%s", header='mu_lin, r_lin, on/off, Vt')

    return total_return

In [15]:
# Displays status of averaging, which is calculated from and appended to the end of output filename

@app.callback(
    Output('average', 'children'),
    [Input('average-data', 'filename')]
)
def average(filename):
    df = np.genfromtxt(filename)
    average = np.mean(df, axis=0)
    outputs = np.vstack((df, average))
    
    np.savetxt(filename, outputs, delimiter=" ", fmt="%s", header='mu_lin, r_lin, on/off, Vt')
    
    return 'Averages have been appended to the last row of "', filename, '"' 

In [ ]:
# Opens browser page to host dashboard

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/dash/resources.py:75: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.ly/external-resources

127.0.0.1 - - [19/Jan/2020 09:55:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2020 09:55:42] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_2_2m1578894578.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2020 09:55:42] "GET /_dash-component-suites/dash_renderer/react@16.v1_2_2m1578894578.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2020 09:55:42] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_2_2m1578894578.7.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2020 09:55:42] "GET /_dash-component-su

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/sanjeev/

127.0.0.1 - - [19/Jan/2020 09:55:43] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/npyio.py", line 1764, in genfromtxt
    fhd = iter(fid)
TypeError: 'NoneType' object is not iterable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/

...

127.0.0.1 - - [19/Jan/2020 09:55:43] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/sanjeev/

127.0.0.1 - - [19/Jan/2020 09:55:43] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/sanjeev/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/sanjeev/

127.0.0.1 - - [19/Jan/2020 09:55:43] "POST /_dash-update-component HTTP/1.1" 500 -
